In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real, X_pred):
    y_pred = dnn_model(X_pred)
    return tf.math.subtract(y_pred, 42)

In [7]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [8]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[-1.1699982]], pop: [[40.83]]
epoch: 1, loss: [[-1.0043068]], pop: [[40.995693]]
epoch: 2, loss: [[-1.1517029]], pop: [[40.848297]]
epoch: 3, loss: [[-1.0434074]], pop: [[40.956593]]
epoch: 4, loss: [[-1.1947708]], pop: [[40.80523]]
epoch: 5, loss: [[-1.2048874]], pop: [[40.795113]]
epoch: 6, loss: [[-1.0598869]], pop: [[40.940113]]
epoch: 7, loss: [[-1.2188034]], pop: [[40.781197]]
epoch: 8, loss: [[-1.35112]], pop: [[40.64888]]
epoch: 9, loss: [[-1.3115921]], pop: [[40.688408]]
epoch: 10, loss: [[-1.3646355]], pop: [[40.635365]]
epoch: 11, loss: [[-1.2594833]], pop: [[40.740517]]
epoch: 12, loss: [[-1.5510902]], pop: [[40.44891]]
epoch: 13, loss: [[-1.3467674]], pop: [[40.653233]]
epoch: 14, loss: [[-1.3822632]], pop: [[40.617737]]
epoch: 15, loss: [[-1.3915253]], pop: [[40.608475]]
epoch: 16, loss: [[-1.5804939]], pop: [[40.419506]]
epoch: 17, loss: [[-1.5821838]], pop: [[40.417816]]
epoch: 18, loss: [[-1.5192337]], pop: [[40.480766]]
epoch: 19, loss: [[-1.541565]],

In [9]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [12]:
print(pop_result.max())
print(pop_result.min())
print(pop_result.mean())

40.995693
28.761856
29.570704


In [10]:
df = np.append(X_result, pop_result, axis=1)

In [11]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic.xlsx')